In [1]:
import pandas as pd
import numpy as np

import sys
sys.path.append('../../pycore/')
from simulate import RBA_result
from utils import extract_details_from_rxnid

In [2]:
df_cases = pd.read_csv('./input/data_list.txt', sep='\t')
cases = df_cases.data_id.to_list()
df_cases.index = df_cases.data_id.to_list()

In [3]:
df_compile = pd.DataFrame(columns=cases)

for case in cases:
    path_data = './datasets/' + case + '/'
    biom_id = df_cases.biomass_dilution_id[case]
    mu = df_cases.growth_rate[case]
    
    # Get kapp (1/s)
    path = path_data + 'kapps_in_vivo.txt'
    df_kapp = pd.read_csv(path, sep='\t')
    
    for i in df_kapp.index:
        _,rxn,rdir,enz = extract_details_from_rxnid(df_kapp.rxnid[i])
        regid = rxn + '_' + rdir
        df_compile.loc[regid, case] = df_kapp.loc[i,'kapp (1/s)']

In [4]:
df_compile['id'] = df_compile.index.to_list()

In [5]:
df_rxnenz = pd.read_excel('./input/rxn_enz_pathway.xlsx')
for i in df_rxnenz.index:
    rxn = df_rxnenz.rxn_id[i]
    rdir = df_rxnenz.direction[i]
    entry = rxn + '_' + rdir
    if entry in df_compile.index:
        df_compile.loc[entry, 'subsystem_group'] = df_rxnenz.loc[i, 'subsystem_group']
        df_compile.loc[entry, 'subsystem'] = df_rxnenz.loc[i, 'subsystem']

In [6]:
idx_excl = ['ACALDCD_c_FWD', 'GLYtps_e_REV', 'CERH124A_r_FWD', 'CERH126A_r_FWD',
            'DUTPDP_c_FWD', 'SERtps_e_REV', 'AKGCITta_m_FWD']
idx = [i for i in df_compile.index if i not in idx_excl]
df_compile = df_compile.loc[idx,:]

In [7]:
cols = df_compile.columns.to_list()
df_compile = df_compile.loc[:, cols[-3:] + cols[:-3]]

In [8]:
df_save = pd.read_excel('./results/kapp_inPerSec.xlsx')
df_save.index = df_save.id.to_list()

for rxn in df_save.index:
    for case in cases:
        if rxn in df_compile.index and case in df_compile.columns:
            df_save.loc[rxn,case] = df_compile.loc[rxn,case]